<a href="https://colab.research.google.com/github/Tessellate-Imaging/monk_v1/blob/master/study_roadmaps/4_image_classification_zoo/Classifier%20-%20Plant%20Pathology%20FGVC7%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## Install Monk


## Using pretrained model for classifying plant diseases using images


## Training a classifier from scratch

# Install Monk

  - git clone https://github.com/Tessellate-Imaging/monk_v1.git

  - cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt
        (Select the requirements file as per OS and CUDA version)


In [ ]:
! git clone https://github.com/Tessellate-Imaging/monk_v1.git

In [ ]:
# If using Colab install using the commands below
! cd monk_v1/installation/Misc && pip install -r requirements_colab.txt

# If using Kaggle uncomment the following command
#! cd monk_v1/installation/Misc && pip install -r requirements_kaggle.txt

# Select the requirements file as per OS and CUDA version when using a local system or cloud
#! cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt



# Used trained classifier for demo

In [ ]:
# Import monk

import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Download trained weights

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pnX7BpisfNGIPJFS-M6_jwNOO6SNdSnP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pnX7BpisfNGIPJFS-M6_jwNOO6SNdSnP" -O cls_plantpathology_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq cls_plantpathology_trained.zip 

In [ ]:
ls workspace/Project-Plant-Pathology

In [ ]:
# Keras project
from keras_prototype import prototype

In [ ]:
# Load project in inference mode

gtf = prototype(verbose=1);
gtf.Prototype("Project-Plant-Pathology", "Keras-resnet50", eval_infer=True);

#Other trained models - uncomment 
#gtf.Prototype("Project-Plant-Pathology", "Keras-resne101", eval_infer=True);

In [ ]:
#Infer

In [ ]:
img_name = "workspace/test/1.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/2.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/3.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/4.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

# Training custom classifier from scratch

## Dataset
    - Credits: https://data.mendeley.com/datasets/4drtyfjtfy/1

## Download

In [ ]:
! pip install kaggle

In [ ]:
! kaggle competitions download -c plant-pathology-2020-fgvc7

In [ ]:
! unzip -qq plant-pathology-2020-fgvc7.zip -d plant_pathology

## Update training csv file

In [ ]:
import os
import sys

import pandas as pd
df = pd.read_csv("plant_pathology/train.csv")

combined = [];
from tqdm.notebook import tqdm
for i in tqdm(range(len(df))):
    img_name = df["image_id"][i] + ".jpg";
    if(df["healthy"][i]):
        label = "healthy";
    elif(df["multiple_diseases"][i]):
        label = "multiple_diseases";
    elif(df["rust"][i]):
        label = "rust";
    else:
        label = "scab";
    
    combined.append([img_name, label]);
    
df2 = pd.DataFrame(combined, columns = ['ID', 'Label']) 
df2.to_csv("plant_pathology/train_updated.csv", index=False);

## Training

In [ ]:
# Monk
import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Using mxnet-gluon backend 
#from gluon_prototype import prototype

# For pytorch backend
#from pytorch_prototype import prototype

# For Keras backend
from keras_prototype import prototype

In [ ]:
# Create Project and Experiment

gtf = prototype(verbose=1);
gtf.Prototype("Project-Plant-Pathology", "Keras-resnet101");

In [ ]:
gtf.Default(dataset_path="plant_pathology/images",
            path_to_csv="plant_pathology/train_updated.csv",
            model_name="resnet101", 
            freeze_base_network=False,
            num_epochs=2);

### How to change hyper parameters and models 
  - Docs - https://github.com/Tessellate-Imaging/monk_v1#4
  - Examples - https://github.com/Tessellate-Imaging/monk_v1/tree/master/study_roadmaps/1_getting_started_roadmap

In [ ]:
#Start Training
gtf.Train();

#Read the training summary generated once you run the cell and training is completed

## Testing on a new dataset for submission

In [ ]:
# Import monk

import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Using mxnet-gluon backend 
#from gluon_prototype import prototype

# For pytorch backend
#from pytorch_prototype import prototype

# For Keras backend
from keras_prototype import prototype

In [ ]:
# Create Project and Experiment

gtf = prototype(verbose=1);
gtf.Prototype("Project-Plant-Pathology", "Keras-resnet101", eval_infer=True);

In [ ]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from scipy.special import softmax
df = pd.read_csv("plant_pathology/sample_submission.csv")

In [ ]:
for i in tqdm(range(len(df))):
    img_name = "plant_pathology/images/" + df["image_id"][i] + ".jpg";
    
    #Invoking Monk's nferencing engine inside a loop
    predictions = gtf.Infer(img_name=img_name, return_raw=True);
    out = predictions["raw"]
    
    df["healthy"][i] = out[0];
    df["multiple_diseases"][i] = out[1];
    df["rust"][i] = out[2];
    df["scab"][i] = out[3];

In [ ]:
df.to_csv("submission.csv", index=False);